In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


# Import Libraries

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import nltk
import nltk.corpus
import re
import string
from nltk.corpus import stopwords
import gensim

# Load Data

In [3]:
train_data=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_data=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
submission=pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

# Data preprocessing

In [4]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_data=train_data.drop(["keyword","location"],axis=1)
test_data=test_data.drop(["keyword","location"],axis=1)

train_data.head(2)

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1


In [6]:
type(train_data)

pandas.core.frame.DataFrame

In [7]:
total_data= pd.concat([train_data,test_data],axis=0,ignore_index=True)

In [8]:
total_data

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1.0
1,4,Forest fire near La Ronge Sask. Canada,1.0
2,5,All residents asked to 'shelter in place' are ...,1.0
3,6,"13,000 people receive #wildfires evacuation or...",1.0
4,7,Just got sent this photo from Ruby #Alaska as ...,1.0
...,...,...,...
10871,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,NaN
10872,10865,Storm in RI worse than last hurricane. My city...,NaN
10873,10868,Green Line derailment in Chicago http://t.co/U...,NaN
10874,10874,MEG issues Hazardous Weather Outlook (HWO) htt...,NaN


In [9]:
# handel urls
def remove_url(text):
    url=re.compile(r'https?://S+|www\.\S+')
    return url.sub(r'',text)
def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)

In [10]:
total_data['text']=total_data['text'].apply(lambda x:remove_url(x))
total_data['text']=total_data['text'].apply(lambda x:remove_html(x))

In [11]:
# convert to lower alphabates
for i in range(len(total_data)):
    total_data['text'][i].lower()

In [12]:
stop_words=set(stopwords.words("english"))
print(stop_words)

{"doesn't", "you're", 'm', 'nor', "you've", 'did', 'which', 'are', 'some', "you'll", 'as', "wasn't", 'those', 'that', 'not', 'against', 'from', 'more', 'don', 'hasn', "it's", 'i', 'a', 'here', 'only', 'than', 'having', 'who', 'such', 'they', "she's", 'to', 'be', "didn't", 'needn', 's', 'we', 'through', 'she', 't', 'shouldn', 'up', 'this', 'by', 'shan', 'should', 'any', 'yours', 'with', 'or', 'very', "hadn't", 'under', 'myself', 'were', 'about', 'him', 'of', 'no', 'will', 'out', 'further', "shan't", 'after', 'off', "should've", "aren't", 'own', 'll', 'on', 'other', 'before', 'all', 'is', 'y', "shouldn't", 'am', "mustn't", 'ours', 'wasn', 'mightn', 'do', 'itself', 'his', 'these', 'ourselves', 'below', 'yourselves', 'most', 'couldn', 'into', 'same', 'where', 'you', 'few', 'o', 'hadn', 'each', 'but', 'my', 'over', 'theirs', 'can', 'in', "that'll", 'has', 'during', 'between', 'ain', 'too', 'himself', 'was', 'been', 'd', "wouldn't", "don't", 'yourself', "needn't", "you'd", 'an', 'when', 'bot

In [13]:
#tokenize text
total_data['tokenized_text']=total_data['text'].apply(gensim.utils.simple_preprocess)

In [14]:
total_data.head()

,id,text,target,tokenized_text
0,1,Our Deeds are the Reason of this #earthquake M...,1.0,"[our, deeds, are, the, reason, of, this, earth..."
1,4,Forest fire near La Ronge Sask. Canada,1.0,"[forest, fire, near, la, ronge, sask, canada]"
2,5,All residents asked to 'shelter in place' are ...,1.0,"[all, residents, asked, to, shelter, in, place..."
3,6,"13,000 people receive #wildfires evacuation or...",1.0,"[people, receive, wildfires, evacuation, order..."
4,7,Just got sent this photo from Ruby #Alaska as ...,1.0,"[just, got, sent, this, photo, from, ruby, ala..."


In [15]:
# remove stopwords
def remove_stopwords(s):
    cleantext=[]
    for i in range(len(s)): 
        if(s[i] not in stop_words):
                cleantext.append(s[i])
    return cleantext


In [16]:
total_data['clean_tok_text']=total_data['tokenized_text'].apply(lambda x:remove_stopwords(x))
total_data.head()

,id,text,target,tokenized_text,clean_tok_text
0,1,Our Deeds are the Reason of this #earthquake M...,1.0,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv..."
1,4,Forest fire near La Ronge Sask. Canada,1.0,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]"
2,5,All residents asked to 'shelter in place' are ...,1.0,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o..."
3,6,"13,000 people receive #wildfires evacuation or...",1.0,"[people, receive, wildfires, evacuation, order...","[people, receive, wildfires, evacuation, order..."
4,7,Just got sent this photo from Ruby #Alaska as ...,1.0,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi..."


In [17]:
total_data.shape

(10876, 5)

In [18]:
# join clean tokenized text
def join_fun(text):
    string=" "
    ans=string.join(text)
    return ans
total_data['final_text']=total_data['clean_tok_text'].apply(lambda x:join_fun(x))
total_data.head()

,id,text,target,tokenized_text,clean_tok_text,final_text
0,1,Our Deeds are the Reason of this #earthquake M...,1.0,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",deeds reason earthquake may allah forgive us
1,4,Forest fire near La Ronge Sask. Canada,1.0,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",forest fire near la ronge sask canada
2,5,All residents asked to 'shelter in place' are ...,1.0,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o...",residents asked shelter place notified officer...
3,6,"13,000 people receive #wildfires evacuation or...",1.0,"[people, receive, wildfires, evacuation, order...","[people, receive, wildfires, evacuation, order...",people receive wildfires evacuation orders cal...
4,7,Just got sent this photo from Ruby #Alaska as ...,1.0,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi...",got sent photo ruby alaska smoke wildfires pou...


# Predictions using Count Vectorizer

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

vec=cv.fit_transform(total_data["final_text"])

In [20]:
print(total_data.shape,vec.shape)

(10876, 6) (10876, 28319)


In [21]:
X=vec[:len(train_data)]
y=train_data.target
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.90)

In [22]:
model=RandomForestClassifier(n_estimators=65)
model.fit(X_train,y_train)
pred=model.predict(X_test)
accuracy_score(pred,y_test)

0.7795275590551181

# Using TfIdf Vectorizer
TF-IDF is better than Count Vectorizers because it not only focuses on the frequency of words present in the corpus but also provides the importance of the words. We can then remove the words that are less important for analysis, hence making the model building less complex by reducing the input dimensions.


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvec=TfidfVectorizer()

vec1=tfidfvec.fit_transform(total_data["final_text"])

In [24]:
X=vec1[:len(train_data)]
y=train_data.target
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.90)

In [25]:
model=RandomForestClassifier(n_estimators=65)
model.fit(X_train,y_train)
pred=model.predict(X_test)
accuracy_score(pred,y_test)


0.8018372703412073

In [26]:
test_vac=vec[len(train_data):]
test_vac

<3263x28319 sparse matrix of type '<class 'numpy.int64'>'
	with 35775 stored elements in Compressed Sparse Row format>

In [27]:
predictios=model.predict(test_vac)
dataframe=pd.DataFrame(predictios,columns=["target"])
submission['target']=dataframe
submission.to_csv('submission.csv',index=False)